In [1]:
import numpy as np
import pandas as pd

In [2]:
breakdowns = pd.read_csv('desolationofeden_kickguitar.csv')

In [3]:
breakdowns.drop(breakdowns.columns[[0, 1]], axis=1, inplace=True)

In [4]:
X = breakdowns.fillna(-1)

In [5]:
X.shape

(1280, 4)

### Get "Vocab" (number of unique row configurations)

In [6]:
def get_vocab(X):
    full_vocab = []
    vocab = []
    indices_vocab = dict()
    vocab_indices = dict()
    for row in X.as_matrix():
        full_vocab.append(list(row))
        if list(row) not in vocab:
            vocab.append(list(row))
            indices_vocab.update({vocab.index(list(row)): list(row)})
            vocab_indices.update({str(list(row)): vocab.index(list(row))})
    return full_vocab, vocab, indices_vocab, vocab_indices

In [7]:
full_vocab, vocab, indices_vocab, vocab_indices = get_vocab(X)

### Split into Semi-Redundant Sequences of Fixed Length

In [8]:
maxlen = 32
step = 4
sequences = []
next_beat = []

In [9]:
for i in range(0, len(full_vocab)-maxlen, step):
    sequences.append(full_vocab[i: i+maxlen])
    next_beat.append(full_vocab[i + maxlen])

##### Helper Function from fchollet/keras/blob/master/examples/lstm_text_generation.py

In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Vectorization

In [11]:
X = np.zeros((len(sequences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sequences), len(vocab)), dtype=np.bool)

In [12]:
for i, measures in enumerate(sequences):
    for t, beat in enumerate(measures):
        X[i, t, vocab_indices[str(beat)]] = 1
    y[i, vocab_indices[str(next_beat[i])]] = 1

### Model

In [13]:
import sys
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [14]:
model = Sequential()

model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(len(vocab)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Train Model

In [17]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = np.random.randint(0, len(full_vocab) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = []
        sequence = full_vocab[start_index: start_index + maxlen]
        generated.append(sequence)
        
        for i in range(256):
            x = np.zeros((1, maxlen, len(vocab)))
            for t, beat in enumerate(sequence):
                x[0, t, vocab_indices[str(beat)]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_beat = indices_vocab[next_index]

            generated.append(next_beat)
            sequence = sequence[1:]
            sequence.append(next_beat)
            
            sys.stdout.write(str(next_beat)+'\n')
            sys.stdout.flush()
        print()   


--------------------------------------------------
Iteration 1
Epoch 1/1
312/312 [==============================] - 1s - loss: 2.6791     

----- diversity: 0.2
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[-1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0]
[0.0, 3.0, 3.0, 3.0]
[0.0, 3.0, -1.0, 3.0]
[1.0, 0.0, 0.0, 0.5]
[1.0, 0.0, 0.0, 0.0]
[-1.0, 0.0, -1.0, 3.0]
[0.0, 3.0, 3.0, 3.0]
[0.0, 3.0, -1.0, 3.0]
[0.0, 3.0, -1.0, 4.0]
[0.0, 3.0, -1.0, 3.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[1.0, 0.0, 0.0, 0.5]
[1.0, 0.0, 0.0, 0.5]
[-1.0, 0.0, 0.0, 0.0]
[0.0, 3.0, -1.0, 3.0]
[1.0, 0.0, 0.0, 0.5]
[-1.0, 3.0, -1.0, 4.0]
[0.0, 3.0, -1.0, 3.0]
[0.0, 4.0, 4.0, 0.0]
[0.0, 3.0, -1.0, 3.0]
[-1.0, 0.0, -1.0, 3.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0]
[0.0, 3.0, -1.0, 3.0]
[-1.0, 3.0, -1.0, 4.0]
[1.0, 0.0, 0.0, 0.5]
[0.0, 0.0, 0.0, 3.0]
[1.0, -1.0, -1.0, -1.0]
[0.0, 3.0, -

[0.0, 0.0, 0.0, 3.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, 3.0, 3.0, 3.0]
[0.0, 3.0, 3.0, 0.0]
[1.0, 0.0, 0.0, 0.5]
[-1.0, 0.0, -1.0, 3.0]
[-1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 3.0]
[0.0, 3.0, -1.0, 4.0]
[-1.0, 0.0, -1.0, 3.0]
[-1.0, 0.0, -1.0, 3.0]
[-1.0, 0.0, -1.0, 3.0]
[0.0, 0.0, -1.0, 3.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 1.0, 1.0, 3.0]
[-1.0, 3.0, -1.0, 4.0]
[1.0, -1.0, -1.0, -1.0]
[0.0, 3.0, -1.0, 3.0]
[-1.0, 3.0, -1.0, 4.0]
[-1.0, 3.0, -1.0, 4.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 0.0, -1.0, 2.0]
[1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 4.0, 4.0, 0.0]
[1.0, 0.0, 0.0, 0.5]
[0.0, 3.0, -1.0, 4.0]
[0.0, 3.0, -1.0, 3.0]
[-1.0, 0.0, 0.0, 0.0]
[0.0, 3.0, -1.0, 3.0]
[0.0, 3.0, -1.0, 4.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 4.0, 4.0, 0.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 3.0, 3.0, 0.0]
[-1.0, 3.0, -1.0, 4.0]
[0.0, 3.0, -1.0, 4.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, 3.0, 3.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, -1.0, -1.0, -1.0]
[0.0, 3.0, -1.0, 4.0]

[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 3.0, -1.0, 4.0]
[0.0, 0.0, -1.0, 3.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, -1.0, -1.0, -1.0]
[1.0, 0.0, 0.0, 0.0]
[-1.0, 3.0, -1.0, 4.0]
[0.0, 1.0, 1.0, 3.0]
[1.0, 0.0, 0.0, 0.5]
[0.0, 4.0, 4.0, 0.0]
[-1.0, 0.0, 0.0, 0.0]
[0.0, 3.0, -1.0, 4.0]
[0.0, 4.0, 4.0, 0.0]
[0.0, 3.0, -1.0, 4.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 1.0, 0.0, 1.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 0.0]
[-1.0, 3.0, -1.0, 4.0]
[0.0, 3.0, -1.0, 3.0]
[1.0, 0.0, 0.0, 0.0]


----- diversity: 1.2
[0.0, 1.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 1.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 3.0, -1.0, 3.0]
[1.0, 0.0, 0.0, 0.5]
[0.0, 3.0, -1.0, 3.0]
[-1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0]
[0.0, 4.0, 4.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, 3.0, -1.0, 4.0]
[0.0, 1.0, 1.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, 0.0, -1.0, 3.0]
[0.0, 1.0, 0.0, 1.0]
[0.0, 1.0, 

[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.

[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 3.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 0.0, 0.0, 3.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]


----- diversity: 1.0
[0.0, 3.0, 3.0, 3.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[1.0, 0.0, 0.0, 0.5]
[-1.0, 0.0, -1.0, 3.0]
[0.0, 3.0, 3.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 0.0]
[0

...

[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 1.0, 3.0]
[1.0, 1.0, 1.0, 0.5]
[1.0, 1.0, 1.0, 0.5]
[0.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 4.0, 4.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[

[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, 1.0, 1.0, 3.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -

[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]
[0.0, -1.0, -1.0, -1.0]


--------------------------------------------------
Iteration 59
Epoch 1/1
312/312 [==============================] - 0s - loss: 0.1451     

----- diversity: 0.2
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.

[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 0.0, -1.0, 2.0]
[0.0, 0.0, -1.0, 2.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[

[-1.0, -1.0, -1.0, -1.0]
[0.0, 0.0, -1.0, 2.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, 3.0, -1.0, 4.0]
[0.0, 1.0, 1.0, 0.0]
[0.0, 1.0, 0.0, 1.0]
[0.0, -1.0, -1.0, -1.0]
[-1.0, 0.0, -1.0, 3.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, -1.0]
[0.0, 1.0, 0.0, 1.0]
[-1.0, -1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0, 